In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
'''
input_size：x的特征维度
hidden_size：隐藏层的特征维度
num_layers：lstm隐层的层数，默认为1
bias：False则bih=0和bhh=0. 默认为True
batch_first：True则输入输出的数据格式为 (batch, seq, feature)
dropout：除最后一层，每一层的输出都进行dropout，默认为: 0
bidirectional：True则为双向lstm默认为False
输入：input, (h0, c0)
输出：output, (hn,cn)
输入数据格式：
input(seq_len, batch, input_size)
h0(num_layers * num_directions, batch, hidden_size)
c0(num_layers * num_directions, batch, hidden_size)

输出数据格式：
output(seq_len, batch, hidden_size * num_directions)
hn(num_layers * num_directions, batch, hidden_size)
cn(num_layers * num_directions, batch, hidden_size)

'''

In [7]:
class RNN(nn.Module):
    def __init__(self,in_dim,hidden_dim,n_layer,n_class,bidirectional=False):
        super(RNN,self).__init__()
        self.n_layer = n_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(in_dim,hidden_dim,n_layer,bidirectional=bidirectional,batch_first=True)
        self.direction = 2 if bidirectional else 1
        self.classifier = nn.Linear(hidden_dim * self.direction,n_class)
    def forward(self,x):
        out,(h_n,c_n) = self.lstm(x)
        res = self.classifier(out)
        print(out.shape,h_n.shape,c_n.shape,res.shape)

In [8]:
rnn = RNN(10,20,1,3,True)
x = torch.randn((5,3,10))
rnn(x)

torch.Size([5, 3, 40]) torch.Size([2, 5, 20]) torch.Size([2, 5, 20]) torch.Size([5, 3, 3])


In [ ]:
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
#in_channels(int) – 输入信号的通道。在文本分类中，即为词向量的维度；图像中为RGB三通道。
#out_channels(int) – 卷积产生的通道。有多少个out_channels，就需要多少个1维卷积
#kernel_size(int or tuple) - 卷积核的尺寸，卷积核的大小
#stride(int or tuple, optional) - 卷积步长
#padding (int or tuple, optional)- 输入的每一条边补充0的层数
#dilation(int or tuple, `optional``) – 卷积核元素之间的间距
#groups(int, optional) – 从输入通道到输出通道的阻塞连接数
#bias(bool, optional) - 如果bias=True，添加偏置

In [50]:
class TextCNN(nn.Module):
    def __init__(self,vocab_size,embedding_size,num_class,kernel_num=100,kernel_sizes=(2,3,4),drop_out=0.5,is_train=True):
        super(TextCNN,self).__init__()
        self.emd = nn.Embedding(vocab_size,embedding_size)
        self.convs = nn.ModuleList([nn.Conv2d(1,kernel_num,(kernel_size,embedding_size)) for kernel_size in kernel_sizes])
        self.drop_out = nn.Dropout(drop_out)
        self.classifier = nn.Linear(len(kernel_sizes) * kernel_num,num_class)
        self.is_train = is_train
    def forward(self,x):
        x = self.emd(x)
        x = x.unsqueeze(1)
        print(x.size())
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        print(x[0].shape,x[1].shape)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) 
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             
             i in x] #[(N,Co), ...]*len(Ks)
        print(x[0].shape,x[1].shape)
        concated = torch.cat(x,1)
        if self.is_train:
            concated = self.drop_out(concated)
        logits = self.classifier(concated)
        return logits

In [51]:
textCnn = TextCNN(100,50,5)
seq = [[1,2,3,4,5],[5,4,3,2,1],[6,7,8,9,7],[7,9,7,8,6]]
seqs = [seq for i in range(3)]

In [52]:
seqs = torch.tensor(seqs,dtype=int)
print(seqs[0].shape)
textCnn(seqs[0])

torch.Size([4, 5])
torch.Size([4, 1, 5, 50])
torch.Size([4, 100, 4]) torch.Size([4, 100, 3])
torch.Size([4, 100]) torch.Size([4, 100])


tensor([[-0.0302, -0.7218, -0.7472, -0.1455,  1.8735],
        [ 0.0666, -0.3853, -0.3979, -0.1382,  0.5298],
        [-0.5417, -0.3284, -0.4055,  0.3855,  1.2028],
        [ 0.0988, -0.0505, -0.2216,  0.0774,  0.4394]],
       grad_fn=<AddmmBackward>)